In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.preprocessing import StandardScaler
import os


class InverseNeuralNet(pl.LightningModule):
    def __init__(self, layer_sizes, lr = 0.01, lr_factor = 0.0):
        super(InverseNeuralNet, self).__init__()
        
        modules = []
        for i in range(len(layer_sizes) - 1):
            modules.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            
            if i != len(layer_sizes) - 2:
                modules.append(nn.ReLU())
        
        self.forward_prop = nn.Sequential(*modules)
        self.learning_rate = lr
        self.lr_factor = lr_factor
        self.save_hyperparameters()
        
    def forward(self, x):
        return self.forward_prop(x)
    
    def training_step(self, batch, batch_idx):
        dos, params = batch
        
        # Forward pass
        predicted = self(dos)
        loss = F.mse_loss(predicted, params)
        
        #log to tensorboard
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        dos, params = batch
        
        # Forward pass
        predicted = self(dos)
        loss = F.mse_loss(predicted, params)
        
        #log to tensorboard
        self.log("val_loss", loss)
        return loss
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        if self.lr_factor == 0.0:
            return optimizer
        
        else:
            optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
            sch = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=self.lr_factor, min_lr = 1e-7)
            return {
                "optimizer":optimizer,
                "lr_scheduler" : {
                    "scheduler" : sch,
                    "monitor" : "train_loss",

                }
            }
        
        
        
class InverseDataset(Dataset):

    def __init__(self, dos_arr, params_arr):
        
        self.dos_arr = torch.from_numpy(dos_arr).float()
        self.params_arr = torch.from_numpy(params_arr).float()
    
    def __getitem__(self, index):
        return self.dos_arr[index], self.params_arr[index]
    
    def __len__(self):
        return self.params_arr.size(dim=0)
    
    
class InverseDataModule(pl.LightningDataModule):
    def __init__(self, data_loc, batch_size):
        super().__init__()
        self.data = ScaledData(data_loc)
        self.batch_size = batch_size
    
    def setup(self, stage: str = None):
        
        if stage == "fit" or stage is None:
            self.train_dataset = InverseDataset(self.data.train_dos, self.data.train_params)
            self.val_dataset = InverseDataset(self.data.val_dos, self.data.val_params)
            
        if stage == "test" or stage is None:
            self.test_dataset = DosDataset(self.data.test_dos, self.data.test_params)
            
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, num_workers=2)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=2)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size = self.batch_size, num_workers=2)
    

class ScaledData:
    def __init__(self, data_loc):
        self.train_set = np.load(os.path.join(data_loc, 'train-set.npz'))
        self.train_params = self.train_set['params']
        self.train_dos_unscaled = self.train_set['dos']

        self.val_set = np.load(os.path.join(data_loc, 'val-set.npz'))
        self.val_params = self.val_set['params']
        self.val_dos_unscaled = self.val_set['dos']

        self.test_set = np.load(os.path.join(data_loc, 'test-set.npz'))
        self.test_params = self.test_set['params']
        self.test_dos_unscaled = self.test_set['dos']

        #With standard scaling
        scaler = StandardScaler()
        self.train_dos = scaler.fit_transform(self.train_dos_unscaled)
        self.val_dos = scaler.transform(self.val_dos_unscaled)
        self.test_dos = scaler.transform(self.test_dos_unscaled)


def plot_one(ax, mse_params, index, see_baseline):
    ax.set_ylim([-0.5, 1.0])
    tick_pos = np.arange(0, 3)
    ax.plot(tick_pos, mse_params[index][2], label = "Ground Truth")
    ax.plot(tick_pos, mse_params[index][1], label = "ML Predicted")
    
    if see_baseline:
        ax.plot(tick_pos, [0, 0, 0.6], label = "baseline")
    
    ax.set_xticks(tick_pos, ('t1', 't2', 'J'))
    ax.legend()

def see_results(predicted, truth, grid_shape, percentiles, see_baseline = False):
    mse_mat = (predicted - truth) ** 2
    mse_list = np.mean(mse_mat, axis = 1)
    print(f"model mse: {np.mean(mse_list)}")

    mse_params = zip(mse_list, predicted, truth)
    mse_params = sorted(mse_params, key = lambda x: x[0], reverse = True)
    
    dim1, dim2 = grid_shape
    fig, ax = plt.subplots(dim1, dim2, figsize = (15, dim1 * 5))
    
    for i in range(dim1):
        for j in range(dim2):
            if i * dim2 + j < len(percentiles):
                percentile = percentiles[i * dim2 + j]
                index = percentile * (len(mse_params)//100)
                plot_one(ax[i][j], mse_params, index, see_baseline)
                ax[i][j].set_title(f"{percentile} percentile")

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
import os
import sys
from sklearn.preprocessing import StandardScaler
import pickle
root = "/project/wyin/jlee/ml-project/"
util_loc = os.path.join(root, "utils")
sys.path.append(util_loc)

#########Code to make hparam iterable#################
architectures = []
for n_layers in range (1, 7):
    #Uniform architecture
    n_neurons = 8
    while n_neurons <= 256:
        layer_sizes = [n_neurons] * n_layers
        n_neurons *= 2
        architectures.append(layer_sizes)
    
    #Linear architecture
    if n_layers > 1:
        start_neurons = 32
        while start_neurons <= 256:
            layer_sizes = []
            for i in range(n_layers, 0, -1):
                layer_sizes.append(i * (start_neurons // n_layers))

            start_neurons *= 2
            architectures.append(layer_sizes)
    
    #Exponential architecture
    if n_layers > 2:
        start_neurons = 32
        while start_neurons <= 256:
            layer_sizes = [start_neurons]
            for i in range(n_layers - 1):
                layer_sizes.append(layer_sizes[len(layer_sizes) - 1] // 2)

            start_neurons *= 2

            architectures.append(layer_sizes)

for layer_sizes in architectures:
    layer_sizes.insert(0, 354)
    layer_sizes.append(3)
######################################################

# step_num = int(sys.argv[1])
# num_cpus = int(sys.argv[2])
# torch.set_num_threads(num_cpus // 2)

#######Set changed hyperparemter(s) here###############
layer_sizes = [354, 256, 256, 256, 3]
learning_rate = 0.001
batch_size = 256
schedule_factor = 0.5
max_time = "00:00:00:30"

data_loc = os.path.join(root, "inverse-shifted/data")
save_loc = os.path.join(root, "inverse-shifted/neural-nets/hyperparameters")
log_name = "arch-search-3-test"

######################################################
log_path = os.path.join(save_loc, "logs", log_name)
val_path = os.path.join(save_loc, "val-ends", log_name)

# if step_num == 1:
#     os.mkdir(log_path)
#     os.mkdir(val_path)
#     print("created paths")

dos_data = InverseDataModule(data_loc, batch_size)
logger = TensorBoardLogger(log_path, name = f'{layer_sizes},{learning_rate},{batch_size},{schedule_factor}')
trainer = pl.Trainer(enable_checkpointing=False, max_time=max_time, logger = logger, enable_progress_bar = True)
model = InverseNeuralNet(layer_sizes, lr = learning_rate, lr_factor = schedule_factor)
trainer.fit(model, datamodule=dos_data)


end_res = trainer.validate(model, dataloaders = dos_data)
f = open(f"{val_path}/{layer_sizes},{learning_rate},{batch_size},{schedule_factor}","wb")
pickle.dump(end_res,f)
f.close()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type       | Params
--------------------------------------------
0 | forward_prop | Sequential | 223 K 
--------------------------------------------
223 K     Trainable params
0         Non-trainable params
223 K     Total params
0.893     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/wyin/.conda/envs/ML-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/wyin/.conda/envs/ML-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Time limit reached. Elapsed time is 0:00:30. Signaling Trainer to stop.


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss          1.9760531358770095e-05
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


FileNotFoundError: [Errno 2] No such file or directory: '/project/wyin/jlee/ml-project/inverse-shifted/neural-nets/hyperparameters/val-ends/arch-search-3-test/[354, 256, 256, 256, 3],0.001,256,0.5'

In [20]:
print(dos_data.train_dataset.dos_arr)

train_dos = dos_data.train_dataset.dos_arr.detach().numpy()

tensor([[ 0.0000, -0.0108, -0.0234,  ..., -0.0272, -0.0125,  0.0000],
        [ 0.0000, -0.0108, -0.0234,  ..., -0.0272, -0.0125,  0.0000],
        [ 0.0000, -0.0108, -0.0234,  ..., -0.0272, -0.0125,  0.0000],
        ...,
        [ 0.0000, -0.0108, -0.0234,  ..., -0.0272, -0.0125,  0.0000],
        [ 0.0000, -0.0108, -0.0234,  ..., -0.0272, -0.0125,  0.0000],
        [ 0.0000, -0.0108, -0.0234,  ..., -0.0272, -0.0125,  0.0000]])


In [21]:
print(np.sum(train_dos, axis = 0))

[ 0.00000000e+00 -1.54751223e-02 -1.42881423e-02  1.38205662e-02
  1.29438937e-03 -1.97360814e-02 -1.12810284e-02  1.74804032e-02
  3.67304683e-03  9.30932164e-03  1.42600238e-02  6.04927540e-03
  3.44485044e-03  1.53815150e-02  6.65807724e-03 -1.81096792e-03
  2.04628706e-03  1.14739537e-02  3.68493795e-03  4.68838215e-03
  2.45356560e-03 -7.77125359e-04  1.19763613e-02 -9.93967056e-03
  3.61800194e-04  3.19242477e-03 -2.54166126e-03 -1.02458000e-02
 -8.59063864e-03 -4.79137897e-03  3.19612026e-03  1.16688013e-03
 -6.38782978e-03  5.45293093e-03 -1.41739845e-04  2.35950947e-03
 -1.60205364e-03 -4.54962254e-04  2.76446342e-04  1.84917450e-03
 -2.12824345e-03 -7.51137733e-04  4.05907631e-05 -1.52841210e-03
 -2.09450722e-04  9.28640366e-05  1.12533569e-04 -3.00616026e-04
 -6.94304705e-04 -3.27944756e-04 -8.55028629e-05  2.06828117e-05
  2.53021717e-05  1.62124634e-05  9.06586647e-05 -9.04500484e-05
  1.86890364e-04 -8.76188278e-05  1.34795904e-04 -1.55746937e-04
  1.89900398e-04 -1.38580